# 确立测试环境
## 1. 建立 Datum 文件夹
       ### 1.1 文件夹包含 vipdoc 下的选出的  lc1 ,lc5,day
       > ** build table in sqlite, : recent statistics **
           * stockcode, date, cci ,max_min_vol,trend regression, recent_support, pressure 
       ### 1.2  包含抓取的1分钟数据，多个股票的混合数据

In [2]:
import sys
sys.path.append("..")
import stockdata 
import pandas as pd
import numpy as np
dat_path = "../datum/"  #where hold the test used data set

In [3]:
SimuDatumHq = pd.DataFrame()
MarketDatum = pd.DataFrame()
filename = 'hq1daymix.csv'
fullname = dat_path + filename
SimuDatumHq = pd.read_csv(fullname)
global counter
counter = 0

In [4]:
# Turn Tick Data to Dictory Data structure 
cols_all =  ['stockcode','open', 'close','price','high', 'low','buy1','buy2','vol', 'amount',
           'B1V','B1', 'B2V','B2','B3V','B3','B4V','B4', 'B5V','B5','S1V',
           'S1', 'S2V','S2', 'S3V','S3','S4V','S4','S5V','S5','DateTime' ] 
cols_use =  ['stockcode','open', 'close','price','high', 'low','vol', 'DateTime']

In [5]:
SimuDatumHq.head()

,stockname,Dealdetails,Timestamp
0,sz002241,"27.93, 28.03, 27.93, 27.93, 27.93, 27.9, 27.93...",2014-08-22 09:29:31
1,sz300024,"32.81, 32.81, 32.63, 32.81, 32.53, 32.56, 32.6...",2014-08-22 09:30:04
2,sz002241,"27.93, 28.03, 27.95, 27.95, 27.89, 27.95, 27.9...",2014-08-22 09:30:20
3,sz300070,"30.58, 30.51, 30.52, 30.58, 30.5, 30.51, 30.52...",2014-08-22 09:30:56
4,sz300024,"32.81, 32.81, 32.4, 32.81, 32.4, 32.4, 32.42, ...",2014-08-22 09:31:05


In [6]:
SimuDatumHq.tail()

,stockname,Dealdetails,Timestamp
930,sz002130,"10.92, 10.92, 10.89, 11.05, 10.77, 10.89, 10.9...",2014-08-22 14:56:56
931,sz002241,"27.93, 28.03, 27.86, 28.28, 27.64, 27.87, 27.8...",2014-08-22 14:57:56
932,sz300070,"30.58, 30.51, 31.11, 31.69, 30.2, 31.11, 31.11...",2014-08-22 14:57:59
933,sz300024,"32.81, 32.81, 33.88, 35.68, 32.15, 33.87, 33.8...",2014-08-22 14:57:59
934,sz002130,"10.92, 10.92, 10.9, 11.05, 10.77, 10.9, 10.9, ...",2014-08-22 14:57:59


 * stockname has prefix and stockcode only contain digitals *  
  1. stockname = "sz002001"
  1. stockcode = stockname[2:]
### full line of whole data from Hq.sina
 *  tick_data = [stockcode] + tick_data + [_row.Timestamp]
  ** current used Data **
 *   tick_data[0:4] + tick_data[7] + [_row.Timestamp]
### cols_use =  ['stockcode','open', 'close','price','high', 'low','vol', 'DateTime']

In [7]:
def getStockData(counter,typ):
    """ counter is the location sequence number , 
    typ = full , return full length
    typ = used , return used fields """
    _row = SimuDatumHq.iloc[counter]
    stockcode = _row.stockname[2:]
    line_dat = _row.Dealdetails    
    tick_data = map(float,line_dat.split(','))
    if type == "full":
        out_data = [stockcode] + tick_data + [_row.Timestamp]
    else: 
        out_data = [stockcode]+ tick_data[0:5] + [tick_data[7]] +[_row.Timestamp]
    return out_data

In [8]:
tick_data = getStockData(0,"used")
tick_data

['002241', 27.93, 28.03, 27.93, 27.93, 27.93, 15700.0, '2014-08-22 09:29:31']

In [9]:
import time
#MarketDatum  = pd.DataFrame(columns=cols_all)
MarketDatum  = pd.DataFrame(columns=cols_use)


def Simulate_getting_DataFrame():
    start = time.time()
    counter = 0
    # run for 350 seconds
    while (time.time() - start) < 350 and (counter< 21):
        tickdata = getStockData(counter,"used")
        #print tickdata
        # connect and read data
        #np.datetime
        # dtime = pd.Timestamp(tickdata[-1])
        npary = np.array(tickdata)
        MarketDatum.loc[counter] = npary
        counter += 1
        #print "Running -----%s==>>"%counter
        #time.sleep(1)

Simulate_getting_DataFrame()
MarketDatum.index = MarketDatum["DateTime"] # assign DataFrame index datetime
df = MarketDatum

### Demo DataFrame various usage!!

In [10]:
cols_use[:-1]   # exclude last datatime field
df= df[cols_use[:-1]]  
#np.array?
# df
df = df[df.stockcode == '002241']  #DataFrame of the stockcode 
st_df= df[cols_use[1:7]] # exclude the stockcode
st_df
df = st_df.astype(np.float) # assign float type to numeric fields
df
#df.dtypes
df.sort_values(by='vol', ascending=False)  
#MarketDatum_use = df[cols_use]
#MarketDatum_use

,open,close,price,high,low,vol
DateTime,,,,,,
2014-08-22 09:35:26,27.93,28.03,27.94,27.99,27.81,373000.0
2014-08-22 09:33:29,27.93,28.03,27.85,27.99,27.84,242580.0
2014-08-22 09:32:32,27.93,28.03,27.89,27.99,27.84,141663.0
2014-08-22 09:31:20,27.93,28.03,27.90,27.99,27.84,71380.0
2014-08-22 09:30:20,27.93,28.03,27.95,27.95,27.89,47037.0
2014-08-22 09:29:31,27.93,28.03,27.93,27.93,27.93,15700.0


In [11]:
print df.dtypes
df.sort_index(axis=1)  # return to normal index 
#df.sort_index?

open     float64
close    float64
price    float64
high     float64
low      float64
vol      float64
dtype: object


,close,high,low,open,price,vol
DateTime,,,,,,
2014-08-22 09:29:31,28.03,27.93,27.93,27.93,27.93,15700.0
2014-08-22 09:30:20,28.03,27.95,27.89,27.93,27.95,47037.0
2014-08-22 09:31:20,28.03,27.99,27.84,27.93,27.90,71380.0
2014-08-22 09:32:32,28.03,27.99,27.84,27.93,27.89,141663.0
2014-08-22 09:33:29,28.03,27.99,27.84,27.93,27.85,242580.0
2014-08-22 09:35:26,28.03,27.99,27.81,27.93,27.94,373000.0


### stat table : calculate the fields
#### first load the day and the lc1 files , in the directory ../datum/

In [12]:
stockname = "sz002594"
fullname = dat_path + stockname + ".day"
print fullname
byd_days = stockdata.daysDataOfStock(fullname)

../datum/sz002594.day


In [13]:
bydDF = byd_days.DF
bydDF.tail()

,open,high,low,close,vol
date_idx,,,,,
2016-09-05,58.00,58.26,57.60,57.92,26712.51
2016-09-06,58.12,59.52,57.78,59.51,84732.65
2016-09-07,59.50,59.60,58.90,58.97,51287.28
2016-09-08,58.98,59.25,58.75,58.90,32724.36
2016-09-09,59.03,60.28,58.99,59.01,67664.29


In [14]:
bydDF.head()

,open,high,low,close,vol
date_idx,,,,,
2016-06-20,59.51,60.16,59.28,59.60,44497.36
2016-06-21,59.95,60.60,59.21,59.39,98047.29
2016-06-22,60.42,61.80,60.15,60.92,133208.45
2016-06-23,60.36,60.99,59.77,60.03,65656.58
2016-06-24,59.99,60.49,57.30,59.11,82357.35


### pre definition of the CCI Computing 
### Notice that first calculate the type price 
    #### then the sma of type price for length of the windows


In [22]:
def compute_bars_CCI(bars, window):
    """computing the bars cci from a dataframe data type
    window = 20
    bars = byd_day_DF """
    def dev(bars):

        return (np.abs(bars.typrice - bars.tp_sma[-1]))

    def mdev(bars):
        """mean deviation """
        return np.round(np.sum(dev(bars))/len(bars),2)

    length = len(bars)
    bars["typrice"] = (bars.close + bars.high + bars.low)/3 
    bars["tp_sma"] = bars.typrice.rolling(window).mean()
    
    bars["mdev"] = pd.concat([(pd.Series(mdev(bars.iloc[i:i+window]), \
                            index=[bars.index[i+window-1]])) for i in xrange(length-window)])
    cci = (bars.typrice-bars.tp_sma)/(0.015*bars.mdev) 
    #bars["cci"] =  cci.round(2)
    return cci.round(2)

In [23]:
cci = compute_bars_CCI(bydDF,20)
cci

date_idx
2016-06-20       NaN
2016-06-21       NaN
2016-06-22       NaN
2016-06-23       NaN
2016-06-24       NaN
2016-06-27       NaN
2016-06-28       NaN
2016-06-29       NaN
2016-06-30       NaN
2016-07-01       NaN
2016-07-04       NaN
2016-07-05       NaN
2016-07-06       NaN
2016-07-07       NaN
2016-07-08       NaN
2016-07-11       NaN
2016-07-12       NaN
2016-07-13       NaN
2016-07-14       NaN
2016-07-15    190.82
2016-07-18    176.94
2016-07-19    125.85
2016-07-20    116.85
2016-07-21     66.96
2016-07-22     27.70
2016-07-25    -49.27
2016-07-26    -48.89
2016-07-27   -168.69
2016-07-28   -152.12
2016-07-29   -138.86
2016-08-01   -149.59
2016-08-02   -146.75
2016-08-03   -131.41
2016-08-04   -109.01
2016-08-05    -95.71
2016-08-08   -100.15
2016-08-09    -75.40
2016-08-10    -65.01
2016-08-11    -66.99
2016-08-12    -45.37
2016-08-15     -5.84
2016-08-16     11.44
2016-08-17     72.17
2016-08-18     55.17
2016-08-19     37.35
2016-08-22    -40.46
2016-08-23    -71.88
2016

In [24]:
cci = compute_bars_CCI(bydDF,14)
cci

date_idx
2016-06-20       NaN
2016-06-21       NaN
2016-06-22       NaN
2016-06-23       NaN
2016-06-24       NaN
2016-06-27       NaN
2016-06-28       NaN
2016-06-29       NaN
2016-06-30       NaN
2016-07-01       NaN
2016-07-04       NaN
2016-07-05       NaN
2016-07-06       NaN
2016-07-07    123.46
2016-07-08    157.70
2016-07-11     92.48
2016-07-12     13.49
2016-07-13      3.15
2016-07-14      7.70
2016-07-15    187.46
2016-07-18    159.88
2016-07-19    102.37
2016-07-20     96.47
2016-07-21     37.53
2016-07-22     -5.01
2016-07-25    -84.67
2016-07-26    -77.87
2016-07-27   -161.94
2016-07-28   -131.52
2016-07-29   -114.83
2016-08-01   -124.06
2016-08-02   -118.35
2016-08-03    -98.97
2016-08-04    -79.54
2016-08-05    -69.78
2016-08-08    -80.75
2016-08-09    -58.86
2016-08-10    -49.12
2016-08-11    -59.50
2016-08-12     -2.55
2016-08-15    149.06
2016-08-16    157.05
2016-08-17    208.12
2016-08-18    120.45
2016-08-19     64.18
2016-08-22    -30.83
2016-08-23    -66.96
2016